In [13]:
import os
import os
import gurobipy as gp
from gurobipy import GRB
import time
from itertools import combinations
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import networkx as nx
import pandas as pd

# Data Reading

In [14]:
def read_tsp_file(file_path):
    edge_weight_section = []
    is_edge_weight_section = False

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove extra whitespace
            
            if line.startswith("DIMENSION"):
                num_cities = int(line.split(":")[1].strip())
                print(f"Number of cities: {num_cities}")
                continue
            
            if line.startswith("EDGE_WEIGHT_SECTION"):
                is_edge_weight_section = True
                continue
            
            if line.startswith("EDGE_WEIGHT_FORMAT"):
                format = line.split(":")[1].strip()
                # print(f"Edge weight format: {format}")
                continue

            if is_edge_weight_section:
                if line == "EOF" or line == 'DISPLAY_DATA_SECTION':  # Stop reading if end of file marker is found
                    break

                # Add numbers from the line to the edge_weight_section list
                edge_weight_section.extend(map(int, line.split()))

    return edge_weight_section, num_cities, format






# TSP with OR tools

In [15]:
def solve_tsp_with_ortools(distance_dict, num_cities):
    """
    Solve TSP using Google OR-Tools
    
    Args:
        distance_dict: Dictionary with (i,j) tuples as keys and distances as values
        num_cities: Number of cities in the problem
    
    Returns:
        route: List of cities in optimal order
        total_distance: Total distance of the optimal route
    """
    # Create the routing model
    manager = pywrapcp.RoutingIndexManager(num_cities, 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        
        # Convert to 1-based indexing to match your distance_dict
        from_node += 1
        to_node += 1
        
        # Check both orderings since we might have only half the matrix
        if (from_node, to_node) in distance_dict:
            return distance_dict[(from_node, to_node)]
        elif (to_node, from_node) in distance_dict:
            return distance_dict[(to_node, from_node)]
        return 0  # Should not happen if distance_dict is complete

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic (cheapest addition)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    # Set metaheuristic
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    
    # Set time limit
    search_parameters.time_limit.FromSeconds(10)

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        # Extract the route
        route = []
        total_distance = 0
        index = routing.Start(0)
        
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node + 1)  # Convert back to 1-based indexing
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            total_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        
        # Add the final node to complete the tour
        route.append(route[0])
        
        return route, total_distance
    else:
        return None, None


# Function to create a dictionary with the distances between the cities

In [16]:

# Lower triangular matrix
def create_distance_dict_lower_diag_row(edge_weight_section, num_cities):
    distance_dict = {}
    idx = 0  # To track position in the edge_weight_section

    # Iterate through lower triangular matrix
    for i in range(1, num_cities+1):
        for j in range(1, i+1):
            if i != j:  # Ignore the diagonal elements (distances between same cities)
                distance_dict[(i, j)] = edge_weight_section[idx]
            idx += 1

    return distance_dict


# Upper triangular matrix
def create_distance_dict_upper_diag_row(edge_weight_section, num_cities):
    distance_dict = {}
    index = 0  # To keep track of the position in the edge_weight_section

    # Loop through each city `i`
    for i in range(num_cities):
        # For each city `i`, iterate over `j` from `i` to `num_cities` (inclusive of diagonal)
        for j in range(i, num_cities):
            if i != j:  # Ignore diagonal (i == j) if needed
                distance_dict[(i + 1, j + 1)] = edge_weight_section[index]
            index += 1

    return distance_dict

def create_distance_dict_full_matrix(edge_weight_section, num_cities):
    distance_dict = {}
    index = 0  # To keep track of the position in the edge_weight_section

    # Loop through each city `i`
    for i in range(num_cities):
        # For each city `i`, iterate over all cities `j`
        for j in range(i, num_cities):
            if i != j:  # Ignore diagonal (i == j) if needed
                distance_dict[(i + 1, j + 1)] = edge_weight_section[index]
            index += 1
        index += i + 1

    return distance_dict



# Utility functions for blossom cuts

In [17]:
def find_blossom_cuts(graph, x_vals, tolerance=0):
    """
    Find blossom inequalities by:
    1. Finding cyclic subgraphs with fractional values
    2. Checking edges going out of these cycles
    3. Identifying cycles with odd number of outgoing edges with value 1
    
    Args:
        graph: NetworkX graph
        x_vals: Dictionary of edge values from LP solution {(u,v): value}
        tolerance: Tolerance for considering a value fractional or 1
    """
    def is_fractional(val):
        return tolerance < val < 1 - tolerance
    
    def is_one(val):
        return abs(val - 1.0) < tolerance
    
    def find_fractional_cycles():
        """Find all cycles in the graph that consist of fractional edges"""
        # Create subgraph with only fractional edges
        fractional_edges = [(u, v) for (u, v), val in x_vals.items() 
                          if is_fractional(val)]
        frac_graph = nx.Graph(fractional_edges)
        # print(frac_graph)
        
        # Find all cycles in the fractional graph
        cycles = []
        try:
            cycles = nx.cycle_basis(frac_graph)
        except nx.NetworkXNoCycle:
            pass
        return cycles
    
    def get_outgoing_edges(cycle_nodes):
        """Get all edges that have exactly one endpoint in the cycle"""
        cycle_set = set(cycle_nodes)
        outgoing = []
        
        # Check all edges in the original graph
        for (u, v), val in x_vals.items():
            u_in = u in cycle_set
            v_in = v in cycle_set
            if u_in != v_in:  # Exactly one endpoint in cycle
                outgoing.append((u, v, val))
                
        return outgoing
    
    def count_unit_edges(edges):
        """Count edges that have value 1"""
        return sum(1 for _, _, val in edges if is_one(val))
    
    blossoms = []
    
    # Step 1: Find all cycles with fractional values
    fractional_cycles = find_fractional_cycles()
    
    # Step 2: For each cycle, check outgoing edges
    for cycle in fractional_cycles:
        # Get all edges going out of the cycle
        outgoing_edges = get_outgoing_edges(cycle)
        
        # Count edges with value 1
        unit_edges_count = count_unit_edges(outgoing_edges)
        
        # If odd number of unit edges, we've found a blossom
        if unit_edges_count % 2 == 1:
            # Store the handle (cycle) and teeth (unit edges)
            teeth = [(u, v) for u, v, val in outgoing_edges if is_one(val)]
            blossoms.append({
                'handle': cycle,
                'teeth': teeth,
                'num_teeth': unit_edges_count
            })
    
    return blossoms

def get_delta_edges(node_set, all_edges):
        """Get edges with exactly one endpoint in the given set"""
        delta_edges = []
        node_set = set(node_set)
        for i, j in all_edges:
            i_in = i in node_set
            j_in = j in node_set
            if i_in != j_in:  # Exactly one endpoint in set
                delta_edges.append((i, j))
        return delta_edges

# Utility function for connectivity cuts

In [18]:
# Subtour function
def subtour(edges):
    unvisited = list(range(1, num_cities+1))
    cycle = range(num_cities+1)  # initial length has 1 more city
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*') if j in unvisited]
        if len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

# Utility function for 2 connected graph cuts

In [19]:

def check_min_cut_flow_from_one(selected_edges, edge_weights, num_nodes):
    """
    Check if the graph has any cuts with flow less than 2 from node 1 to all other nodes.
    Parameters:
        selected_edges: list of edges selected in the current solution
        edge_weights: dictionary with edge weights from vals
        num_nodes: total number of nodes in the graph
    Returns:
            - has_cut_less_than_2: boolean indicating if there's a cut < 2
            - cut_value: the value of the minimum cut found
            - cut_partition: the nodes in the smaller partition of the cut
            - violating_sink: the sink node causing the violation
    """
    # Create NetworkX graph
    G = nx.Graph()
    G.add_nodes_from(range(1, num_nodes + 1))
    
    # Add selected edges with their weights
    for i, j in selected_edges:
        weight = edge_weights[i, j]
        if weight > 1e-3:  # Only consider edges with significant weight
            G.add_edge(i, j, capacity=weight)
    
    min_cut_value = float('inf')
    min_cut_partition = None
    violating_sink = None
    
    # Check min cut from node 1 to all other nodes
    source = 1
    for sink in range(2, num_nodes + 1):
        try:
            cut_value, partition = nx.minimum_cut(G, source, sink)
            
            if cut_value < min_cut_value:
                min_cut_value = cut_value
                min_cut_partition = partition
                violating_sink = sink
        except nx.NetworkXError:
            return True, 0, {source}, sink
    
    return (min_cut_value < (2), min_cut_value, 
            set(min_cut_partition[0]) if min_cut_partition else None,
            violating_sink)

def validate_connectivity_from_one(selected_edges, edge_weights, num_nodes):
    """
    Validate if the graph has sufficient connectivity from node 1.
    Parameters:
        selected_edges: list of edges selected in the current solution
        edge_weights: dictionary with edge weights from vals
        num_nodes: Total number of nodes
    Returns:
            - is_valid: boolean indicating if connectivity requirement is met
            - cut_info: details about the violating cut if found
    """
    has_small_cut, cut_value, cut_partition, violating_sink = check_min_cut_flow_from_one(selected_edges, edge_weights, num_nodes)
    
    if has_small_cut:
        cut_info = {
            'cut_value': cut_value,
            'partition1': cut_partition,
            'partition2': set(range(1, num_nodes + 1)) - cut_partition,
            'violating_sink': violating_sink
        }
        # print("cut_info", cut_info)
        return False, cut_info
    
    return True, None

# Function to add conncetivity cuts, 2 connected cuts, blossom cuts

In [20]:
def subtourelim_with_blossom_and_2connected_cuts(model, where):
    global num_cities, lazy_cut_count, user_cut_count

    if where == GRB.Callback.MIPNODE:
        if model.cbGet(GRB.Callback.MIPNODE_STATUS) == GRB.OPTIMAL:
            # Get solution at integer feasible solution
            vals = model.cbGetNodeRel(model._vars)
            selected = gp.tuplelist((i, j) for i, j in model._vars.keys() if vals[i, j] > 1e-3)
            edge_weights = {(i, j): vals[i, j] for i, j in selected}
            # Find the shortest cycle in the selected edge list
            tour = subtour(selected)
            if len(tour) < num_cities:
                # Add subtour elimination constraint if the tour is incomplete
                model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2)) <= len(tour) - 1)
                lazy_cut_count += 1

            else:
                # Check for 2-connected cuts
                is_valid, cut_info = validate_connectivity_from_one(selected, edge_weights, num_cities)
                
                if not is_valid:
                    cut_partition = cut_info['partition1']
                    model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(cut_partition, 2) 
                                             if (i, j) in model._vars.keys()) <= len(cut_partition) - 1)
                    lazy_cut_count += 1
                    # print(2)

                # Check for blossom constraints
                else:
                    graph = nx.Graph()
                    graph.add_edges_from(selected)
                    blossoms = find_blossom_cuts(graph, edge_weights)
                    # print(f"Found {len(blossoms)} blossoms")
                    
                    for blossom in blossoms:
                        handle = blossom['handle']
                        teeth = blossom['teeth']
                        k = len(teeth)

                        expr = 0
                        all_edges = list(model._vars.keys())

                        # Calculate x(δ(H)) for handle
                        handle_delta = get_delta_edges(handle, all_edges)
                        for i, j in handle_delta:
                            if (i, j) in model._vars:
                                expr += model._vars[i, j]

                        # Calculate sum(x(δ(T_i))) for each tooth
                        for tooth in teeth:
                            tooth_endpoints = set([tooth[0], tooth[1]])
                            tooth_delta = get_delta_edges(tooth_endpoints, all_edges)
                            for i, j in tooth_delta:
                                if (i, j) in model._vars:
                                    expr += model._vars[i, j]

                        # Add blossom constraint: x(δ(H)) + sum(x(δ(T_i))) ≥ 3k + 1
                        model.cbCut(expr >= 3 * k + 1)
                        user_cut_count += 1
    elif where == GRB.callback.MIPSOL:
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys() if vals[i, j] > 1e-3)
        tour = subtour(selected)
        if len(tour) < num_cities:
            # Add subtour elimination constraint if the tour is incomplete
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2)) <= len(tour) - 1)
            lazy_cut_count += 1

 



# Call functions to read data, create distance dictionary, create gurobi model, and solve the model

In [21]:
# Function to read and process the TSP instance, set up the model, and solve it
def solve_tsp_instance(file_path, total_time_limit=120):
    global lazy_cut_count, user_cut_count, num_cities

    # Step 1: Read and process the TSP file, and record the time taken
    start_time = time.time()
    edge_weight_section, num_cities, format = read_tsp_file(file_path)
    
    # Initialize distance dictionary based on format
    if format == 'UPPER_DIAG_ROW':
        distance_dict = create_distance_dict_upper_diag_row(edge_weight_section, num_cities)
    elif format == 'LOWER_DIAG_ROW':
        distance_dict = create_distance_dict_lower_diag_row(edge_weight_section, num_cities)
    elif format == 'FULL_MATRIX':
        distance_dict = create_distance_dict_full_matrix(edge_weight_section, num_cities)
    
    matrix_creation_time = time.time() - start_time

    # Calculate remaining time for solving
    remaining_time = total_time_limit - matrix_creation_time
    if remaining_time <= 0:
        print("Matrix creation exceeded total time limit. Skipping instance.")
        return

    # Step 2: Initialize Gurobi model
    m = gp.Model()
    m.setParam(GRB.Param.OutputFlag, 0)
    m.setParam(GRB.Param.Cuts, 0)
    m.setParam(GRB.Param.Heuristics, 0.0)
    m.setParam(GRB.Param.Presolve, 0)
    m.setParam(GRB.Param.TimeLimit, remaining_time)
    m.setParam('MIPGap', 0.0)  # Set MIP gap tolerance
    # Step 3: Define variables and constraints
    vars = m.addVars(distance_dict.keys(), obj=distance_dict, vtype=GRB.BINARY, name='x')
    vars.update({(j, i): vars[i, j] for i, j in vars.keys()})
    m.addConstrs(vars.sum(i, '*') == 2 for i in range(1, num_cities + 1))
    m._vars = vars
    m.Params.lazyConstraints = 1
    
    # Step 4: write lp file
    m.write('model.lp')
    
    # Step 5: copy model for solving without cuts
    model_without_cuts = m.copy()


    lazy_cut_count = 0
    user_cut_count = 0
    solve_start_time = time.time()
    m.optimize(subtourelim_with_blossom_and_2connected_cuts)
    solve_time = time.time() - solve_start_time

    # Step 6: Collect statistics
    best_lp_bound = m.ObjBound
    best_ip_solution = m.ObjVal
    nodes_explored = m.NodeCount
    gap = m.MIPGap

    # Step 7: Optimize without lazy cuts
    model_without_cuts.optimize()
    best_ip_solution_no_cuts = model_without_cuts.ObjVal
    # print(f"Best IP Solution without user/lazy cuts: {best_ip_solution_no_cuts}")

    
    # Step 8: OR-tools solution
    route, total_distance = solve_tsp_with_ortools(distance_dict, num_cities)
    
    # Step 9: Prepare result dictionary
    result = {
        "Problem Class": "STSP",
        "Instance": os.path.basename(file_path),
        "Best LP Bound": best_lp_bound,
        "Best IP solution with cuts": best_ip_solution,
        "Best IP solution without user/lazy cuts": best_ip_solution_no_cuts,
        "Nodes explored (with cuts)": nodes_explored,
        "User cuts added": user_cut_count,
        "Lazy cuts added": lazy_cut_count,
        "GAP": gap,
        "Time": solve_time + matrix_creation_time,
        "Google OR-tools solution": total_distance
    }

    return result



In [30]:
def main():
    results = []  # Initialize an empty list to store each result
    for root, dirs, files in os.walk("STSP"):
        files = sorted(files)
        for file in files:
            # if file.startswith("da"):
                print(f"----------------{file}----------------")
                result = solve_tsp_instance(os.path.join(root, file))
                if result:
                    results.append(result)  # Append each result dictionary to the list
    
    # Create a pandas DataFrame from the results list and save to CSV
    df = pd.DataFrame(results)
    df.to_csv('tsp_results.csv', index=False)
    print("Results saved to tsp_results.csv")
            

In [31]:
if __name__ == "__main__":
    main()

----------------dantzig42.tsp----------------
Number of cities: 42
Results saved to tsp_results.csv
